In [5]:
!pip install pandas scikit-learn




[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# ✅ 3. Import necessary libraries
# data preprocessing means that we will be dealing with data in a way that is suitable for machine learning algorithms
import pandas as pd
from sklearn.model_selection import train_test_split
#Import preprocessing libraries which is used for data preprocessing  
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder 
# pickle is a library that is used to save and load data
import pickle


# ✅ 4. Load the dataset
# load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()
# | RowNumber | CustomerId | Surname  | CreditScore | Geography | Gender | Age | Tenure | Balance   | NumOfProducts | HasCrCard | IsActiveMember | EstimatedSalary | Exited |
# | --------- | ---------- | -------- | ----------- | --------- | ------ | --- | ------ | --------- | ------------- | --------- | -------------- | --------------- | ------ |
# | 1         | 15634602   | Hargrave | 619         | France    | Female | 42  | 2      | 0.00      | 1             | 1         | 1              | 101348.88       | 1      |
# | 2         | 15647311   | Hill     | 608         | Spain     | Female | 41  | 1      | 83807.86  | 1             | 0         | 1              | 112542.58       | 0      |
# | 3         | 15619304   | Onio     | 502         | France    | Female | 42  | 8      | 159660.80 | 3             | 1         | 0              | 113931.57       | 1      |
# | 4         | 15701354   | Boni     | 699         | France    | Female | 39  | 1      | 0.00      | 2             | 0         | 0              | 93826.63        | 0      |
# | 5         | 15737888   | Mitchell | 850         | Spain     | Female | 43  | 2      | 125510.82 | 1             | 1         | 1              | 79084.10        | 0      |

# ✅ Drop irrelevant columns
data.drop(columns=["RowNumber", "CustomerId", "Surname"], inplace=True)
data.head()
# | Index | CreditScore | Geography | Gender | Age | Tenure |   Balance   | NumOfProducts | HasCrCard | IsActiveMember | EstimatedSalary | Exited |
# |-------|-------------|-----------|--------|-----|--------|-------------|----------------|-----------|----------------|------------------|--------|
# |   0   |     619     |  France   | Female |  42 |    2   |    0.00     |       1        |     1     |        1       |    101348.88     |   1    |
# |   1   |     608     |  Spain    | Female |  41 |    1   |  83807.86   |       1        |     0     |        1       |    112542.58     |   0    |
# |   2   |     502     |  France   | Female |  42 |    8   | 159660.80   |       3        |     1     |        0       |    113931.57     |   1    |
# |   3   |     699     |  France   | Female |  39 |    1   |    0.00     |       2        |     0     |        0       |     93826.63     |   0    |
# |   4   |     850     |  Spain    | Female |  43 |    2   | 125510.82   |       1        |     1     |        1       |     79084.10     |   0    |

# ✅ 6. Label Encoding on Gender column
# For Categorical data(Gender,Geography ), we will be using some type of encoding 
labelEncoder_gender = LabelEncoder()
data['Gender'] = labelEncoder_gender.fit_transform(data['Gender']) 
data.head()
# | Index | CreditScore | Geography | Gender | Age | Tenure | Balance   | NumOfProducts | HasCrCard | IsActiveMember | EstimatedSalary | Exited |
# | ----- | ----------- | --------- | ------ | --- | ------ | --------- | ------------- | --------- | -------------- | --------------- | ------ |
# | 0     | 619         | France    | 0      | 42  | 2      | 0.00      | 1             | 1         | 1              | 101348.88       | 1      |
# | 1     | 608         | Spain     | 0      | 41  | 1      | 83807.86  | 1             | 0         | 1              | 112542.58       | 0      |
# | 2     | 502         | France    | 0      | 42  | 8      | 159660.80 | 3             | 1         | 0              | 113931.57       | 1      |
# | 3     | 699         | France    | 0      | 39  | 1      | 0.00      | 2             | 0         | 0              | 93826.63        | 0      |
# | 4     | 850         | Spain     | 0      | 43  | 2      | 125510.82 | 1             | 1         | 1              | 79084.10        | 0      |
# ✅ 7. Problem with Geography column
# For Geography, we will be using one hot encoding because 
# if we 0,1,2 to represent the different geographies, ML think it is ranking so we use one hot encoding  
# which give different values to different geographies

# ✅ 8. Use One-Hot Encoding for Geography
one_hot_encoder_geo = OneHotEncoder()
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded
# <Compressed Sparse Row sparse matrix of dtype 'float64'
# 	with 10000 stored elements and shape (10000, 3)>
# Convert sparse matrix to dense array
geo_encoded_array = geo_encoded.toarray()

# Create a DataFrame from the array with proper column names
geo_encoded_df = pd.DataFrame(
    geo_encoded_array,
    columns=one_hot_encoder_geo.get_feature_names_out(['Geography'])
)

# ✅ 9. Drop old Geography column and add new encoded columns
# Match index with original data
geo_encoded_df.index = data.index

# Drop original 'Geography' column
data.drop(columns=['Geography'], inplace=True)

# Concatenate one-hot encoded columns with original data
data = pd.concat([data, geo_encoded_df], axis=1)

# View updated dataset
data.head()
# | Index | CreditScore | Gender | Age | Tenure | Balance   | NumOfProducts | HasCrCard | IsActiveMember | EstimatedSalary | Exited | Geography_France | Geography_Germany | Geography_Spain |
# | ----- | ----------- | ------ | --- | ------ | --------- | ------------- | --------- | -------------- | --------------- | ------ | ----------------- | ------------------ | ---------------- |
# | 0     | 619         | 0      | 42  | 2      | 0.00      | 1             | 1         | 1              | 101348.88       | 1      | 1.0               | 0.0                | 0.0              |
# | 1     | 608         | 0      | 41  | 1      | 83807.86  | 1             | 0         | 1              | 112542.58       | 0      | 0.0               | 0.0                | 1.0              |
# | 2     | 502         | 0      | 42  | 8      | 159660.80 | 3             | 1         | 0              | 113931.57       | 1      | 1.0               | 0.0                | 0.0              |
# | 3     | 699         | 0      | 39  | 1      | 0.00      | 2             | 0         | 0              | 93826.63        | 0      | 1.0               | 0.0                | 0.0              |
# | 4     | 850         | 0      | 43  | 2      | 125510.82 | 1             | 1         | 1              | 79084.10        | 0      | 0.0               | 0.0                | 1.0              |


# ✅ 10. Save the encoders using Pickle
# Save LabelEncoder for Gender
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(labelEncoder_gender, file)

# Save OneHotEncoder for Geography
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

# ✅ 11. Split dataset into Independent & Dependent Features
# Independent features
X = data.drop(columns=['Exited'])

# Dependent feature
y = data['Exited']

# Check the shapes
print("X shape:", X.shape)
print("y shape:", y.shape)

# X shape: (10000, 12)
# y shape: (10000,)

# ✅ 12. Split into Train and Test sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# ✅ 13. Apply Feature Scaling using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# • Scaling makes all feature values similar
# → Useful especially for algorithms like Neural Networks

# ✅ 14. Save the scaler to a file
# Save the scaler to a file
with open('standard_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)







X shape: (10000, 12)
y shape: (10000,)


### ANN Implementation

In [ ]:
# !pip install tensorflow

import tensorflow as tf
print(tf.__version__) # 2.19.0

# 🔹 Step 1: Import Required Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
import datetime
# •	📦 tensorflow: Main deep learning framework
# •	🏗️ Sequential: To stack layers sequentially
# •	🧱 Dense: Fully connected layer
# •	🎛️ EarlyStopping, TensorBoard: For training control and visualization
# •	🕒 datetime: To timestamp log folders

# 🔹 Step 2: Build and Compile the Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],))) # HL1 connected with input layer
model.add(Dense(32, activation='relu')) # HL2 we don’t need to give input_shape because in Sequential it already know it connected 
model.add(Dense(1, activation='sigmoid'))#only 1 output layer with activation function is sigmoid 

# 2.19.0
# c:\Python311\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
#   super().__init__(activity_regularizer=activity_regularizer, **kwargs)

model.summary()
# Model: "sequential_3"
#  Total params: 2,945 (11.50 KB)
#  Trainable params: 2,945 (11.50 KB)
#  Non-trainable params: 0 (0.00 B)

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
# ┃ Layer (type)                    ┃ Output Shape           ┃ Param #       ┃
# ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
# │ dense_3 (Dense)                 │ (None, 64)             │ 832           │
# ├─────────────────────────────────┼────────────────────────┼───────────────┤
# │ dense_4 (Dense)                 │ (None, 32)             │ 2,080         │
# ├─────────────────────────────────┼────────────────────────┼───────────────┤
# │ dense_5 (Dense)                 │ (None, 1)              │ 33            │
# └─────────────────────────────────┴────────────────────────┴───────────────┘

#compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy", # loss function is binary_crossentropy which is used for binary classification
    metrics=['accuracy'] # accuracy is a metric to measure how good the model is
)

# 🔹 Step 3: Create Log Directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 🗂️ This creates a log folder like: logs/fit/20250731-142530
# •	Helps you differentiate runs
# •	Keeps logs neatly organized

# 🔹 Step 4: Create TensorBoard Callback
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)
# •	🧪 histogram_freq=1: Logs weight histograms every epoch
# •	📊 Enables real-time visualization in TensorBoard
# •	📊 tensorboard_callback is used to visualize the model

# 🔹 Step 5: Set Up EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',        # Watch validation loss
    patience=10,                # Wait 10 epochs before stopping
    restore_best_weights=True # Reload best model
)
# 🧠 Why use it?
# •	Saves training time
# •	Avoids overfitting
# •	Keeps best version of model

# 🔹 Step 6: Train the Model with Callbacks
# X_train,X_test,y_train,y_test
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100, # Number of epochs is used to train the model
    callbacks=[tensorboard_callback, early_stop]
)
# ✅ Benefits:
# •	Training will stop early if no improvement
# •	All logs will be stored and visualized

# Epoch 1/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 3ms/step - accuracy: 0.8136 - loss: 0.4391 - val_accuracy: 0.8395 - val_loss: 0.3815
# Epoch 2/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 2ms/step - accuracy: 0.8453 - loss: 0.3720 - val_accuracy: 0.8535 - val_loss: 0.3527
# Epoch 3/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 2ms/step - accuracy: 0.8565 - loss: 0.3495 - val_accuracy: 0.8535 - val_loss: 0.3459
# Epoch 4/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 2ms/step - accuracy: 0.8594 - loss: 0.3421 - val_accuracy: 0.8625 - val_loss: 0.3460
# Epoch 5/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 2ms/step - accuracy: 0.8614 - loss: 0.3367 - val_accuracy: 0.8600 - val_loss: 0.3457
# Epoch 6/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 2ms/step - accuracy: 0.8618 - loss: 0.3346 - val_accuracy: 0.8620 - val_loss: 0.3426
# Epoch 7/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 3ms/step - accuracy: 0.8640 - loss: 0.3305 - val_accuracy: 0.8605 - val_loss: 0.3433
# Epoch 8/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 3ms/step - accuracy: 0.8629 - loss: 0.3294 - val_accuracy: 0.8580 - val_loss: 0.3429
# Epoch 9/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8618 - loss: 0.3281 - val_accuracy: 0.8615 - val_loss: 0.3464
# Epoch 10/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8652 - loss: 0.3247 - val_accuracy: 0.8585 - val_loss: 0.3419
# Epoch 11/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8652 - loss: 0.3234 - val_accuracy: 0.8670 - val_loss: 0.3386
# Epoch 12/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8655 - loss: 0.3229 - val_accuracy: 0.8640 - val_loss: 0.3393
# Epoch 13/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 3ms/step - accuracy: 0.8639 - loss: 0.3216 - val_accuracy: 0.8620 - val_loss: 0.3428
# Epoch 14/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8655 - loss: 0.3184 - val_accuracy: 0.8555 - val_loss: 0.3436
# Epoch 15/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 3ms/step - accuracy: 0.8681 - loss: 0.3166 - val_accuracy: 0.8595 - val_loss: 0.3382
# Epoch 16/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 3ms/step - accuracy: 0.8684 - loss: 0.3157 - val_accuracy: 0.8590 - val_loss: 0.3403
# Epoch 17/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 3ms/step - accuracy: 0.8670 - loss: 0.3148 - val_accuracy: 0.8585 - val_loss: 0.3382
# Epoch 18/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8677 - loss: 0.3137 - val_accuracy: 0.8590 - val_loss: 0.3402
# Epoch 19/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 5ms/step - accuracy: 0.8677 - loss: 0.3128 - val_accuracy: 0.8545 - val_loss: 0.3460
# Epoch 20/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8689 - loss: 0.3116 - val_accuracy: 0.8605 - val_loss: 0.3454
# Epoch 21/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 7ms/step - accuracy: 0.8687 - loss: 0.3101 - val_accuracy: 0.8575 - val_loss: 0.3420
# Epoch 22/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 6ms/step - accuracy: 0.8686 - loss: 0.3089 - val_accuracy: 0.8565 - val_loss: 0.3453
# Epoch 23/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 5ms/step - accuracy: 0.8702 - loss: 0.3089 - val_accuracy: 0.8630 - val_loss: 0.3395
# Epoch 24/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8710 - loss: 0.3080 - val_accuracy: 0.8590 - val_loss: 0.3445
# Epoch 25/100
# [1m250/250[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 4ms/step - accuracy: 0.8721 - loss: 0.3063 - val_accuracy: 0.8555 - val_loss: 0.3442

# 🔹 Step 7: Save the Trained Model
model.save('model.h5')
# 💾 Saves the entire model (architecture + weights)
# Can be loaded later using:
model = tf.keras.models.load_model('model.h5')










2.19.0


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8000 - loss: 0.4569 - val_accuracy: 0.8290 - val_loss: 0.3957
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8453 - loss: 0.3874 - val_accuracy: 0.8570 - val_loss: 0.3537
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8544 - loss: 0.3563 - val_accuracy: 0.8630 - val_loss: 0.3417
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8595 - loss: 0.3451 - val_accuracy: 0.8600 - val_loss: 0.3409
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8614 - loss: 0.3378 - val_accuracy: 0.8595 - val_loss: 0.3442
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8612 - loss: 0.3340 - val_accuracy: 0.8580 - val_loss: 0.3439
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8665 - loss: 0.3301 - val_accuracy: 0.8670 - val_loss: 0.3379
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8645 - loss: 0.3286 - val_accu

In [ ]:
# 🔹 Step 8: Load TensorBoard in Notebook
%load_ext tensorboard 
%tensorboard --logdir logs/fit
# # 📊 This starts TensorBoard in your Jupyter notebook
# # 🛠️ If using a specific run:
# %tensorboard --logdir logs/fit/20250731-174609

# 🔹 Step 9: Understanding TensorBoard Graphs
# 🧠 Key Charts You’ll See:
# 🔍 Metric	         📈 Meaning
# epoch_accuracy	Model's accuracy per epoch
# epoch_loss	    Training loss per epoch
# val_accuracy	    Validation accuracy (generalization check)
# val_loss	        Validation loss (used by EarlyStopping)
# Histograms	    Weight/bias distributions layer-wise
#biases	            Weights of the dense layer
# epochslearningrate	Learning rate per epoch
# ✅ Good sign:
# •	📉 Loss goes down
# •	📈 Accuracy goes up


# 🧠 What is an Epoch in Machine Learning (especially Deep Learning)?
# An epoch is one complete pass through all the training data during model training.

# ✅ Example to Understand:
# Suppose you have:

# x_train = 1000 training samples
# batch_size = 100
# ➡️ One epoch = model sees all 1000 samples once, usually in 10 batches of 100.


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 15716), started 0:02:42 ago. (Use '!kill 15716' to kill it.)